# Klasyfikacja Fashion-MNIST

Teraz Twoja kolej na zbudowanie i wytrenowanie sieci neuronowej. Będziesz korzystać z [zestawu danych Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist), zastępującego zbiór danych MNIST. Klasyfikacja zbioru MNIST przy wykorzystaniu NN jest zadaniem trywialnym, w którym można łatwo osiągnąć dokładność lepszą niż 97%. Fashion-MNIST to zestaw obrazów ubrań w skali szarości o wymiarach 28x28. Jest bardziej złożony niż MNIST, więc lepiej odzwierciedla rzeczywistą wydajność sieci i lepiej przedstawia zestawy danych, używane w świecie rzeczywistym. 

<img src='assets/fashion-mnist-sprite.png' width=500px>

W tym notatniku zbudujesz własną sieć neuronową. W większości przypadków możesz po prostu skopiować i wkleić kod z ZMUM 6, ale takie postępowanie nie poprawi Twojej umiejętności tworzenia modeli. Ważne jest, aby samodzielnie napisać kod i sprawić by zadziałał. Warto na pewno zapoznania się z poprzednimi notatnikami podczas pracy nad tym.

Na początku załadujmy zbiór danych przez torchvision. 

In [2]:
import torch
from torch import nn
from torchvision import datasets, transforms

import helper

# Zdefiniowanie przekształceń w celu normalizacji danych
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Pobranie i wcztanie zbioru treningowego
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Pobranie i wcztanie zbioru testowego
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

ModuleNotFoundError: No module named 'torch'

Tutaj może modejrzeć jedno ze zdjęć.

In [ ]:
image, label = next(iter(trainloader))
helper.imshow(image[0, :]);

## Budowanie sieci

W poniższym bloku należy zdefiniować architekturę swojej sieci. Podobnie jak w przypadku MNIST, każdy obraz ma wymiary 28x28, co daje w sumie 784 piksele i należy do 1 z 10 klas. Należy utworzyć co najmniej jedną ukrytą warstwę. Sugeruję użycie funkcji aktywacji ReLU dla warstw ukrytych i zwrócenie na wyjściu logitów lub log-softmax. Od Ciebie zależy, ile dodasz warstw oraz ich rozmiar. 

In [ ]:
# TODO: Zdefiniuj architekturę sieci 
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 128),
                      nn.ReLU(),
                      nn.Linear(128, 10),
                      nn.LogSoftmax(dim=1))

# Trenowanie sieci

Teraz należy opracować własną sieć i ją wytrenować. Najpierw trzeba zdefiniować [kryterium](http://pytorch.org/docs/master/nn.html#loss-functions) (coś jak `nn.CrossEntropyLoss`) i [optymalizator](http://pytorch.org/docs/master/optim.html) (zazwyczaj `optim.SGD` lub `optim.Adam`).

Następnie napisz kod uczący:

* Przepuszczenie obrazów przez sieć w celu uzyskania logitów
* Użycie logitów, do wyznaczenia straty
* Wykonaj propagacji wstecznej za pomocą `loss.backward()`, w celu wyznaczenia gradientów
* Wykonanie kroku optymalizatorem, w celu aktualizacji wagi

Dostosowując hiperparametry (ukryte warstwy, tempo uczenia się itp.), powinno uzyskać się `trening loss` poniżej 0,4. 

In [ ]:
from torch import optim

# TODO: Utwórz sieć, zdefiniuj kryterium i optymalizator 
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01)

In [3]:
# TODO: Trenowanie sieci
epochs = 10
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss / len(trainloader)}")

NameError: name 'trainloader' is not defined

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import helper

dataiter = iter(testloader)
images, labels = next(dataiter)
img = images[0]
img = img.resize_(1, 784)

# TODO: Oblicz prawdopodobieństwa klas (f. softmax) dla img 
with torch.no_grad():
    logps = model(img)

ps = torch.exp(logps)

helper.view_classify(img.resize_(1, 28, 28), ps, version='Fashion')

# Inferencja i Walidacja

Teraz, gdy masz wytrenowaną sieć, możesz jej używać do przewidywania. Zwykle nazywa się to **inferencją**/**wnioskowaniem**, terminem zapożyczonym ze statystyk. Jednak sieci neuronowe mają tendencję do *zbyt dobrego* działania na danych uczących i nie są w stanie uogólniać danych, których wcześniej nie widziały. Nazywa się to **przeuczeniem** *(ang. overfitting)* i ma negatywny wpływ na wydajność wnioskowania. Aby przetestować nadmierne dopasowanie podczas treningu, mierzymy wydajność na danych, które nie znajdują się w zestawie treningowym, zwanym zestawem **walidacyjnym**. Nadmiernego dopasowania unikamy poprzez zastosowanie regularyzacji, jak np. dropout oraz poprzez monitorowanie wydajności walidacji podczas treningu. W tym notatniku zobaczymy, jak to zrobić w PyTorch.

Jak zwykle zacznijmy od załadowania zestawu danych przez torchvision. Tym razem skorzystamy z zestawu testowego, który można uzyskać, ustawiając `train=False` tutaj:

```python
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
```

Zestaw testowy zawiera obrazy, podobnie jak zestaw uczący. Zazwyczaj 10-20% oryginalnego zestawu danych przeznacza się do testowania i walidacji, a reszta wykorzystywana jest do trenowania.

Celem walidacji jest zmierzenie wydajności modelu na danych, które nie są częścią zestawu szkoleniowego. Wydajność tutaj zależy jednak od programisty. Zazwyczaj jest to tylko dokładność *(ang. accuracy)*, czyli procent klas, które sieć przewidziała poprawnie. Inne opcje to [precyzja i czułość](https://en.wikipedia.org/wiki/Precision_and_recall#Definition_(classification_context)) oraz wskaźnik błędów 5 największych *(ang. top-5 error rate)*. W tym zadaniu skoncentrujemy się na dokładności. Najpierw zrobimy propagację w przód jedną partią danych testowych.

In [ ]:
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 128),
                      nn.ReLU(),
                      nn.Linear(128, 10),
                      nn.LogSoftmax(dim=1))
images, labels = next(iter(testloader))
images = images.view(images.shape[0], -1)
with torch.no_grad():
    ps = torch.exp(model(images))
print(ps.shape)

Z prawdopodobieństw możemy uzyskać najbardziej prawdopodobną klasę za pomocą metody `ps.topk`. Zwraca to najwyższe wartości $k$. Ponieważ chcemy tylko najbardziej prawdopodobnej klasy, możemy użyć `ps.topk(1)`. Zwraca krotkę wartości górnych $k$ i indeksów górnych $k$. Jeśli najwyższą wartością jest piąty element, otrzymamy 4 jako indeks. 

In [ ]:
top_p, top_class = ps.topk(1, dim=1)
print(top_class[:10, :])

Teraz możemy sprawdzić czy przewidywane klasy pasują do etykiet. Można to łatwo zrobić, porównując `top_class` i `labels`, ale musimy uważać na kształty. Tutaj `top_class` to tensor 2D o kształcie `(64, 1)`, podczas gdy `labels` to 1D z kształtem `(64)`. Aby równość działała tak, jak chcemy, `top_class` i `labels` muszą mieć ten sam kształt.

Jeśli zrobimy

```python
equals = top_class == labels
```

`equals` będzie miał kształt `(64, 64)`, spróbuj samodzielnie. W efekcie zwraca porównanie jednego elementu w każdym wierszu `top_class` z każdym elementem `labels`, co zwraca 64 wartości logiczne True/False dla każdego wiersza.

In [ ]:
equals = top_class.squeeze() == labels
equals

Teraz musimy obliczyć procent poprawnych prognoz. `equals` przyjmuje wartości binarne: 0 lub 1. Oznacza to, że jeśli po prostu zsumujemy wszystkie wartości i podzielimy przez liczbę wartości, otrzymamy procent poprawnych przewidywań. Jest to ta sama operacja, co wyznaczanie średniej, więc dokładność możemy uzyskać za pomocą wywołania `torch.mean`. Jeśli jednak spróbujesz `torch.mean(equals)`, dostaniesz błąd

```
RuntimeError: mean is not implemented for type torch.ByteTensor
```

Dzieje się tak, ponieważ `equals` ma typ `torch.ByteTensor`, a `torch.mean` nie jest zaimplementowany dla tensorów tego typu. Musimy więc przekonwertować `equals` na float tensor. Zauważ, że kiedy bierzemy `torch.mean` zwraca on tensor skalarny, aby uzyskać rzeczywistą wartość jako zmiennoprzecinkową, musimy wykonać `accuracy.item()`.

In [ ]:
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item() * 100}%')

Sieć która nie została wytrenowana zwraca nam wyniki losowe, a jej dokładność powinna wynosić około 10%. Teraz wytrenujmey naszą sieć i dołączając proces walidacji, w celu zmierzenia jak dobrze sieć działa na zestawie testowym. Ponieważ nie aktualizujemy parametrów sici w trakcie walidacji, możemy przyspieszyć wykonanie kodu, wyłączając gradienty za pomocą `torch.no_grad()`:

```python
# wyłącz gradienty
with torch.no_grad():
    # kod walidacyjny
    for images, labels in testloader:
        ...
```

>**Ćwiczenie:** Zaimplementuj poniższą pętlę walidacji i wydrukuj całkowitą dokładność po pętli. Możesz w dużej mierze skopiować i wkleić powyższy kod, ale sugeruję wpisanie go, ponieważ napisanie go samodzielnie jest niezbędne do budowania umiejętności. Ogólnie rzecz biorąc, zawsze dowiesz się więcej, wpisując kod samodzielnie zamiast jego kopiowania i wklejania. Powinno uzyskać się dokładność powyżej 80%. 

In [4]:
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 128),
                      nn.ReLU(),
                      nn.Linear(128, 10),
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 30
steps = 0

for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)

        optimizer.zero_grad()

        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    else:
        with torch.no_grad():
            cum_accuracy = 0
            val_loss = 0
            for images, labels in testloader:
                images = images.view(images.shape[0], -1)

                log_ps = model(images)
                loss = criterion(log_ps, labels)

                val_loss += loss.item()

                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)

                equals = top_class.squeeze() == labels
                cum_accuracy += torch.mean(equals.type(torch.FloatTensor))

            accuracy = (cum_accuracy.item() / len(testloader)) * 100
            print(f'Accuracy: {accuracy:.3f}%')

NameError: name 'nn' is not defined

## Przeuczenie/Overfitting

Jeśli przyjrzymy się stratom związanym z trenowaniem i walidacją podczas trenowania sieci, możemy zauważyć zjawisko zwane przeuczeniem/overfittingiem.

<img src='assets/overfitting.png' width=450px>

Sieć coraz lepiej uczy się zestawu treningowego, co skutkuje mniejszymi stratami treningowymi. Sieć zaczyna jednak mieć problemy z uogólnianiem wiedzy na dane spoza zbioru uczącego, co prowadzi do wzrostu straty na zbiorze walidacyjnym. Ostatecznym celem każdego modelu głębokiego uczenia jest przewidywanie nowych danych, dlatego powinniśmy dążyć do uzyskania jak najmniejszej możliwej straty dla walidacji. Jedną z opcji jest użycie wersji modelu o najmniejszym błędzie walidacji, w naszym modelu około 8-10 epok treningowych. Ta strategia nazywa się *wczesnym zatrzymaniem* *(ang. early-stopping)*. W praktyce często zapisuje się model podczas uczenia, a później wybiera się model o najmniejszym błędzie walidacji.

Najpopularniejszą metodą redukcji overfittingu (poza wczesnym zatrzymaniem) jest *dropout*, gdzie losowo pomijamy neurony. Zmusza to sieć do dzielenia się informacjami między wagami, zwiększając jej zdolność do uogólniania na nowe dane. Dodawanie dropout w PyTorch jest proste dzięki modułowi [`nn.Dropout`](https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout).

```python
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
        # Moduł dropout z prawdopodobieństwem pominięcią równym 0.2 
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        # upewnienie się, że tensor wejściowy jest płaski (wektor danych)
        x = x.view(x.shape[0], -1)
        
        # Tym razem z dropoutem
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        
        # w warstwie wyjściowej nie stosuje się dropoutu
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x
```

Podczas treningu chcemy wykorzystać dropout, aby zapobiec overfittingowi, ale podczas wnioskowania chcemy wykorzystać całą sieć. Dlatego musimy wyłączyć dropout podczas walidacji, testowania i za każdym razem, gdy używamy sieci do prognozowania. Aby to zrobić, użyj `model.eval()`. Ustawia to model w tryb oceny, w którym prawdopodobieństwo dropoutu wynosi 0. Możesz ponownie włączyć dropout, ustawiając model w trybie trenowania za pomocą `model.train()`. Ogólnie rzecz biorąc, wzorzec pętli walidacji w którym wyłączasz gradienty będzie wyglądał tak: ustawiasz model w tryb oceny, obliczasz stratę i metrykę walidacji, a następnie ustawiasz model z powrotem w tryb trenowania. 

```python
# turn off gradients
with torch.no_grad():
    
    # set model to evaluation mode
    model.eval()
    
    # validation pass here
    for images, labels in testloader:
        ...

# set model back to train mode
model.train()
```

> **Ćwiczenie:** Dodaj dropout do swojego modelu i ponownie wytrenuj go w Fashion-MNIST. Sprawdź, czy możesz uzyskać mniejszą stratę walidacji lub wyższą dokładność. 

In [ ]:
## TODO: Zdefiniuj swój model z dodanym przerywaniem
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Dropout(p=0.3),  # Dropout
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Dropout(p=0.3),  # Dropout
    nn.Linear(128, 10),
    nn.LogSoftmax(dim=1)
)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 30

In [ ]:
## TODO: Trenuj swój model z dropoutem i monitoruj postępy w treningu wraz ze  stratą i dokładnością walidacji 
for e in range(epochs):

    model.train()

    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)

        optimizer.zero_grad()

        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    else:
        train_loss = (running_loss / len(trainloader))
        with torch.no_grad():

            model.eval()

            cum_accuracy = 0
            val_loss = 0
            for images, labels in testloader:
                images = images.view(images.shape[0], -1)

                log_ps = model(images)
                loss = criterion(log_ps, labels)

                val_loss += loss.item()

                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)

                equals = top_class.squeeze() == labels
                cum_accuracy += torch.mean(equals.type(torch.FloatTensor))

            test_loss = (val_loss / len(testloader))
            accuracy = (cum_accuracy.item() / len(testloader)) * 100
            print(f'Accuracy: {accuracy:.3f}%')

            print(f"Train loss: {train_loss:10.3f} | Test loss: {test_loss:10.3f}")

## Wnioskowanie

Teraz, gdy model jest wytrenowany, możemy go użyć do wnioskowania. Robiliśmy to już wcześniej, ale teraz musimy pamiętać, aby ustawić model w trybie wnioskowania za pomocą `model.eval()`. Warto również wyłączyć autograd z kontekstem `torch.no_grad()`. 

In [ ]:
# Zaimportuj moduł pomocniczy
import helper

# Przetestuj swoją sieć! 

model.eval()

dataiter = iter(testloader)
images, labels = next(dataiter)
img = images[0]
# Przekształcenie obrazu 2D do wektora 1D
img = img.view(1, 784)

# Wyznaczenie prawdopodobieństw przynależności do klasy (softmax) dla img
with torch.no_grad():
    output = model.forward(img)

ps = torch.exp(output)

# Wyświetl obraz i prawdopodobieństwa 
helper.view_classify(img.view(1, 28, 28), ps, version='Fashion')

# Zapisywanie i ładowanie modeli

Teraz zajmiemy się tym jak zapisywać i ładować modele za pomocą PyTorch. Jest to ważne, ponieważ często chcemy załadować wcześniej wytrenowane modele, aby użyć ich do prognozowania lub kontynuować trenowanie na nowych danych. 

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

import helper
import fc_model

In [ ]:
# Zdefiniowanie przekształceń w celu normalizacji danych
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Pobranie i wcztanie zbioru treningowego
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Pobranie i wcztanie zbioru testowego
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

Podgląd wczytanego obrazka. 

In [5]:
image, label = next(iter(trainloader))
helper.imshow(image[0, :]);

NameError: name 'trainloader' is not defined

# Trenowanie sieci

Aby to było bardziej zwięzłe poprawna architektura modelu i kod uczący został przeniesiony do pliku o nazwie `fc_model`. Importując go, możemy łatwo stworzyć w pełni połączoną sieć za pomocą `fc_model.Network` i trenować sieć za pomocą `fc_model.train`. Możemy użyć tego modelu (po jego nauczeniu), aby zademonstrować, jak możemy zapisywać i ładować modele. 

In [ ]:
# Utwórz sieć, zdefiniuj kryterium i optymalizator 

model = fc_model.Network(784, 10, [512, 256, 128])
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
fc_model.train(model, trainloader, testloader, criterion, optimizer, epochs=2)

## Zapisywanie i ładowanie sieci

Jak można sobie wyobrazić, niepraktyczne jest trenowanie sieci za każdym razem, gdy musimy z niej korzystać. Zamiast tego możemy zapisać wytrenowane modele, a następnie załadować je później, aby trenować je dalej lub użyć ich do prognozowania.

Parametry sieci PyTorch są przechowywane w modelu `state_dict`. Widzimy, że słownik stanu zawiera macierze wag i odchyleń/biasów dla każdej z naszych warstw. 

In [ ]:
print("Nasz model: \n\n", model, '\n')
print("Klucze wektora stanu: \n\n", model.state_dict().keys())

Najprostszą rzeczą do zrobienia jest po prostu zapisanie słownika stanu za pomocą `torch.save`. Na przykład możemy zapisać go do pliku `'checkpoint.pth'`. 

In [6]:
torch.save(model.state_dict(), 'checkpoint.pth')

NameError: name 'torch' is not defined

Następnie możemy załadować słownik stanu za pomocą `torch.load`. 

In [ ]:
state_dict = torch.load('checkpoint.pth')
print(state_dict.keys())

Aby załadować słownik stanu do sieci, wywołujemy `model.load_state_dict(state_dict)`. 

In [ ]:
model.load_state_dict(state_dict)

Wydaje się całkiem proste, ale jak zwykle jest trochę bardziej skomplikowane. Ładowanie słownika stanu działa tylko wtedy, gdy architektura modelu jest dokładnie taka sama jak architektura punktu kontrolnego. Jeśli zbudujemy model o innej architekturze, to operacja ta się nie powiedzie. 

In [7]:
# Spróbuj to
model = fc_model.Network(784, 10, [400, 200, 100])
# Spowoduje to błąd, ponieważ rozmiary tensorów są nieprawidłowe! 
model.load_state_dict(state_dict)

NameError: name 'fc_model' is not defined

Oznacza to, że musimy zbudować model dokładnie taki, jaki był podczas trenowania. Informacje o architekturze modelu muszą być zapisane w punkcie kontrolnym wraz ze stanem. Aby to zrobić, budujemy słownik zawierający wszystkie informacje potrzebne do całkowitego przebudowania modelu. 

In [ ]:
checkpoint = {'input_size': 784,
              'output_size': 10,
              'hidden_layers': [each.out_features for each in model.hidden_layers],
              'state_dict': model.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

Teraz punkt kontrolny ma wszystkie niezbędne informacje do odbudowania wytrenowanego modelu. Można łatwo ustawić tę funkcję. Podobnie możemy napisać funkcję do ładowania punktów kontrolnych. 

In [ ]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = fc_model.Network(checkpoint['input_size'],
                             checkpoint['output_size'],
                             checkpoint['hidden_layers'])
    model.load_state_dict(checkpoint['state_dict'])

    return model

In [ ]:
model = load_checkpoint('checkpoint.pth')
print(model)